# Standard library
* https://pkg.go.dev/std

# archive

# bufio

# builtin

# bytes

In [3]:
// type Ordered interface { ... }

import (
	"cmp"
	"fmt")

%%
fmt.Println(cmp.Compare(1, 2))
fmt.Println(cmp.Compare(2, 1))
fmt.Println(cmp.Compare(1, 1))

-1
1
0


# cmp
* https://pkg.go.dev/cmp

# compress

# container

# context
* https://pkg.go.dev/context
* [context.ipynb](./context.ipynb)

# crypto

# database

# debug

# embed

# encoding

## encoding/json

In [2]:

import (
	"encoding/json"
	"testing"
)

type Movie struct {
	Title  string
	Year   int  `json:"released"`
	Color  bool `json:"color,omitempty"`
	Actors []string
}

var movies = []Movie{
	{Title: "mivie1", Year: 1942, Color: false, Actors: []string{"H1", "I1"}},
	{Title: "mivie2", Year: 1967, Color: true, Actors: []string{"S2", "J2"}},
}

func TestJSON(t *testing.T) {
	// 序列化
	//data, err := json.Marshal(movies)
	data, err := json.MarshalIndent(movies, "", "  ")
	if err != nil {
		t.Errorf("JSON marshaling failed: %s", err)
	}
	t.Logf("%s\n", data)

	// 反序列化
	var titles []struct{ Title string }
	if err := json.Unmarshal(data, &titles); err != nil {
		t.Errorf("JSON Unmarshal failed: %s", err)
	}
	t.Log(titles)
}

%test

=== RUN   TestJSON
    main_test.go:42: [
          {
            "Title": "mivie1",
            "released": 1942,
            "Actors": [
              "H1",
              "I1"
            ]
          },
          {
            "Title": "mivie2",
            "released": 1967,
            "color": true,
            "Actors": [
              "S2",
              "J2"
            ]
          }
        ]
    main_test.go:49: [{mivie1} {mivie2}]
--- PASS: TestJSON (0.00s)
PASS


# errors
* [errors.ipynb](./errors.ipynb)

# expvar

# flag

# fmt

# hash

# html

# image

# index

# io

# iter

# log
* https://pkg.go.dev/log@go1.23.5

## log/slog
structured logging

# maps

# math

# mine

# net

## net/http

In [1]:
import (
	"fmt"
	"log"
	"net/http"
	"sync"
	"testing"
)

var mu sync.Mutex // 信号量, 控制count
var count int     // 访问次数

func init() {
	// log.SetFlags(log.Ldate | log.Ltime | log.Llongfile)
	log.SetFlags(log.Ldate | log.Ltime | log.Lshortfile)
}

func handler(w http.ResponseWriter, r *http.Request) {
	mu.Lock()
	count++
	mu.Unlock()

	fmt.Fprintf(w, "URL.Path=%q\n", r.URL.Path)
	debugHandler(w, r)
}

func debugHandler(w http.ResponseWriter, r *http.Request) {
	fmt.Fprintf(w, "\n\n=== DEBUG === \n\n")
	// method, url, protocol
	fmt.Fprintf(w, "%s %s %s\n", r.Method, r.URL, r.Proto)
	// headers
	for k, v := range r.Header {
		fmt.Fprintf(w, "Header[%q]=%q\n", k, v)
	}
	// host
	fmt.Fprintf(w, "Host=%q\n", r.Host)
	// remote address
	fmt.Fprintf(w, "RemoteAddr=%q\n", r.RemoteAddr)
	// form
	if err := r.ParseForm(); err != nil {
		log.Print(err)
	}
	for k, v := range r.Form {
		fmt.Fprintf(w, "Form[%q]=%q", k, v)
	}
}

func counterHandler(w http.ResponseWriter, r *http.Request) {
	mu.Lock()
	fmt.Fprintf(w, "Count %d\n", count)
	mu.Unlock()

	debugHandler(w, r)
}

func TestHTTP(t *testing.T) {
	http.HandleFunc("/", handler)
	http.HandleFunc("/count", counterHandler)
	log.Fatal(http.ListenAndServe("localhost:9999", nil))
}

%test

=== RUN   TestHTTP


signal: interrupt


# os
* https://pkg.go.dev/os

## os/exec

## os/signal
* https://gobyexample.com/signals

## os/user

# path

# plugin

# reflect
* https://pkg.go.dev/reflect

# regexp
* https://pkg.go.dev/regexp

# runtime

## runtime/cgo
* [tools/_cgo.ipynb](../tools/_cgo.ipynb)

## runtime/coverage

## runtime/debug

## runtime/metrics

## runtime/pprof

## runtime/race

## runtime/trace

# slices

# sort

# strconv

# strings

# structs

# sync
* https://pkg.go.dev/sync

# syscall
* https://pkg.go.dev/syscall

# testing
* https://pkg.go.dev/testing
* [tests.ipynb](./tests.ipynb)

# text

## text/template
* https://pkg.go.dev/text/template

In [3]:
// Github


import (
	"encoding/json"
	"fmt"
	"net/http"
	"net/url"
	"strings"
	"time"
)

const (
	IssuesURL = "https://api.github.com/search/issues"
)

type IssueSearchResult struct {
	TotalCount int `json:"total_count"`
	Items      []*Issue
}

// Issue
type Issue struct {
	Number    int
	HTMLURL   string `json:"html_url"`
	Title     string
	State     string
	User      *User
	CreatedAt time.Time `json:"created_at"`
	Body      string    // Markdown格式
}

// User
type User struct {
	Login   string
	HTMLURL string `json:"html_url"`
}

// SearchIssue
func SearchIssue(terms []string) (*IssueSearchResult, error) {
	q := url.QueryEscape(strings.Join(terms, " "))
	queryUrl := IssuesURL + "?q=" + q
	fmt.Println(queryUrl)
	resp, err := http.Get(queryUrl)
	if err != nil {
		return nil, err
	}

	if resp.StatusCode != http.StatusOK {
		resp.Body.Close()
		return nil, fmt.Errorf("search query failed: %s", resp.Status)
	}

	var result IssueSearchResult
	if err := json.NewDecoder(resp.Body).Decode(&result); err != nil {
		resp.Body.Close()
		return nil, err
	}
	resp.Body.Close()
	return &result, nil
}

In [4]:

import (
	"os"
	"testing"
	"text/template"
	"time"

	// "github.com/zhoujiagen/examples/github"
)

// 展示了模板语言: 动作, 循环, 函数
const textTemplate = `{{.TotalCount}} issues:
{{range .Items}}------------------------------------------------
Number: {{.Number}}
User: {{.User.Login}}
Title: {{.Title | printf "%.64s"}}
Age: {{.CreatedAt | daysAgo}} days
{{end}}`

func daysAgo(t time.Time) int {
	return int(time.Since(t).Hours() / 24)
}

var textReport = template.Must(template.New("issuelist").
	Funcs(template.FuncMap{"daysAgo": daysAgo}).
	Parse(textTemplate))

func TestTemplate(t *testing.T) {
	terms := []string{"HBase"}
	// result, err := github.SearchIssue(terms)
	result, err := SearchIssue(terms)
	if err != nil {
		t.Error(err)
	}

	if err := textReport.Execute(os.Stdout, result); err != nil {
		t.Error(err)
	}
}

%test

=== RUN   TestTemplate
https://api.github.com/search/issues?q=HBase
44456 issues:
------------------------------------------------
Number: 9
User: elliVM
Title: Implement write to HBase 
Age: 3 days
------------------------------------------------
Number: 140
User: stuBirdFly
Title: hbase support cell TTL
Age: 29 days
------------------------------------------------
Number: 7
User: elliVM
Title: Implement SQL to HBase data mapping
Age: 3 days
------------------------------------------------
Number: 6
User: elliVM
Title: Draft WIP: Provide replication feature from Teragrep Mariadb to 
Age: 9 days
------------------------------------------------
Number: 18
User: renovate[bot]
Title: Update dependency org.apache.hbase:hbase-server to v0.99.2
Age: 17 days
------------------------------------------------
Number: 17
User: renovate[bot]
Title: Update dependency org.apache.hbase:hbase-protocol to v0.99.2
Age: 17 days
------------------------------------------------
Number: 135
User: rmdmatting

# time
* https://pkg.go.dev/time

# unicode

# unique

# unsafe
* https://pkg.go.dev/unsafe